In [1]:
import os
import numpy    as np
import pandas   as pd
import psycopg2 as psy

from datetime import datetime as dt
from random   import randrange
from random   import sample
from random   import choice

/usr/local/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
#''' Check if file exists on path. Move existing file to ARCH. Create new input for consultant_dim.'''


In [3]:
# Setup script input parameters

min_obs = 5                      # minimal number of observation in output dataset
max_obs = 20                     # maximal number of observation in output dataset

min_age = 18                     # minimal age of client
max_age = 90                     # maximal age of client

obs = randrange(min_obs,max_obs) # number of observation
obs

13

In [4]:
path_to_arch = "/Users/szymonbocian/Documents/Python/Arch/consultant"
path_to_dir  = "/Users/szymonbocian/Documents/Python/input"
file_name    = "consultant_dim.csv"

now          = dt.now()
path_to_file = os.path.join(path_to_dir, file_name)

if not(os.path.exists(path_to_file) and os.path.isdir(path_to_arch)):
    raise Exception('Incorrect dir')

time_stamp = "_" + str(now)[:10]
arch_file_name = 'arch_' + file_name.replace(".csv",time_stamp + ".csv")
 
os.rename(path_to_file, os.path.join(path_to_arch, arch_file_name))

In [66]:
# connection to database and return list of column column_list
try:
    connection = psy.connect( user = "szymonbocian",
                              password = "",
                              host = "localhost",
                              port = "5432",
                              database = "dwh_call_center")

    cursor = connection.cursor()

    query = "SELECT column_name FROM INFORMATION_SCHEMA.columns WHERE table_schema = 'stg' AND table_name = 'consultant_dim';"
    cursor.execute(query)
    column_list = cursor.fetchall()
    
    query = "SELECT DISTINCT consultant_key FROM pro.consultant_dim WHERE manager_key IS NULL;"
    cursor.execute(query)
    manager_list = cursor.fetchall()
    
except (Exception, psy.Error) as error :
    print ("Error while connecting to PostgreSQL", error)
finally:
        if(connection):
            cursor.close()
            connection.close()

column_list = [''.join(c for c in s if c not in '(),') for s in column_list]

a = list()
for row in manager_list:
    a.append(row[0])

manager_list = a
    
print(column_list)
print(manager_list)

['consultant_first_name', 'consultant_second_name', 'consultant_last_name', 'manager_key', 'consultant_business_key']
[4]


In [60]:
A = ["Adam","Bartłomiej","Cezary","Damian","Erazm","Frank","Grzegorz","Herman","Igor","Jeremi","Karol","Leopold","Maciej","Nikodem","Oskar","Piotr"]
B = ["Agnieszka","Beata","Celina","Dominika","Ewelina","Faustyna","Genowefa","Halina","Iwona","Jadwiga","Karolina","Lucyna","Małgorzata","Natalia","Olga","Patrycja"]
D = ["Astona","Bragiewicz","Cekow","Danicz","Erlang","Fox","Grel","Horn","Iltiempo","Jakarev","Kugelschriber","Longburg","Misha","Nolan","Odin","Piskt"]

In [61]:
df = pd.DataFrame({
    "firstName" : sample(A + B, k = obs),
})

df['isSecName'] = np.random.choice([True, False], obs, True, [0.15, 0.85])
df['isMale']   = df.firstName.isin(A)
df['isFemale'] = df.firstName.isin(B)

df['secondName'] = df.apply(lambda l: choice(A) if l['isMale']   and l['isSecName'] else "", axis = 1)
df['secondName'] = df.apply(lambda l: choice(B) if l['isFemale'] and l['isSecName'] else l['secondName'], axis = 1)

df['lastName'] = sample(D, k = obs)

df['managerKey'] = np.random.choice(manager_list)

,firstName,isSecName,isMale,isFemale,secondName,lastName,managerKey
0,Erazm,False,True,False,,Misha,4
1,Jadwiga,False,False,True,,Piskt,4
2,Damian,False,True,False,,Odin,4
3,Patrycja,False,False,True,,Bragiewicz,4
4,Lucyna,False,False,True,,Kugelschriber,4
5,Maciej,False,True,False,,Danicz,4
6,Adam,False,True,False,,Longburg,4
7,Leopold,False,True,False,,Fox,4
8,Agnieszka,False,False,True,,Cekow,4
9,Natalia,True,False,True,Halina,Grel,4


In [63]:
# Create consultant business key 
fnl = ('00' + (df.firstName.str.len() *8).astype(str))
lnl = ('00' + (df.lastName.str.len() *8).astype(str))
cal = ('00' + df.managerKey.astype(str))

df['clientBusinessKey'] = df.firstName.map(str).str.slice(0,1) + df.lastName.map(str).str.slice(0,1) + [w[-3:] for w in fnl] + [w[-3:] for w in lnl] + [w[-3:] for w in cal] + "E"

,firstName,isSecName,isMale,isFemale,secondName,lastName,managerKey,clientBusinessKey
0,Erazm,False,True,False,,Misha,4,EM040040004E
1,Jadwiga,False,False,True,,Piskt,4,JP056040004E
2,Damian,False,True,False,,Odin,4,DO048032004E
3,Patrycja,False,False,True,,Bragiewicz,4,PB064080004E
4,Lucyna,False,False,True,,Kugelschriber,4,LK048104004E
5,Maciej,False,True,False,,Danicz,4,MD048048004E
6,Adam,False,True,False,,Longburg,4,AL032064004E
7,Leopold,False,True,False,,Fox,4,LF056024004E
8,Agnieszka,False,False,True,,Cekow,4,AC072040004E
9,Natalia,True,False,True,Halina,Grel,4,NG056032004E


In [64]:
df = df.drop(['isSecName', 'isMale', 'isFemale'], axis = 1)
df

,firstName,secondName,lastName,managerKey,clientBusinessKey
0,Erazm,,Misha,4,EM040040004E
1,Jadwiga,,Piskt,4,JP056040004E
2,Damian,,Odin,4,DO048032004E
3,Patrycja,,Bragiewicz,4,PB064080004E
4,Lucyna,,Kugelschriber,4,LK048104004E
5,Maciej,,Danicz,4,MD048048004E
6,Adam,,Longburg,4,AL032064004E
7,Leopold,,Fox,4,LF056024004E
8,Agnieszka,,Cekow,4,AC072040004E
9,Natalia,Halina,Grel,4,NG056032004E


In [65]:
# Create csv file with given destination

df.columns = column_list
df.to_csv(path_to_file, index = None, header = True)